In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\T834432\\Documents\\personnel\\thermostat\\thermostat-agent\\thermostat-292016-05700c0efdbe__.json"
os.environ["PROJECT_ID"] = "thermostat-292016"
os.environ["ACTION_THRESHOLD"] = "1.0"

from google.cloud import storage
#from thermostat import get_metric_list_from_bucket, bucket_name, storage_client
from yadt import parse_date, apply_tz_toronto
from datetime import datetime, timedelta
import pandas as pd
import json
import pickle
import numpy as np

# Instantiates a client
storage_client = storage.Client()
# The name for the new bucket
bucket = storage_client.bucket("thermostat_metric_data")


In [2]:
b = bucket.get_blob('_thermostat_metric_data.p')

pickle_load = b.download_as_bytes()
thermostat_metric_data = pickle.loads(pickle_load)
thermostat_metric_data.sort_index(inplace=True)
#thermostat_metric_data['hash'] = pd.util.hash_pandas_object(thermostat_metric_data)
#thermostat_metric_data = thermostat_metric_data[~thermostat_metric_data.duplicated(keep='first')]
thermostat_metric_data

,location,motion,temperature,humidity,dt,PPD,Indoor Temp. Setpoint,MA Temp.,Htg SP,stove_exhaust_temp,Coil Power,original_payload,temp_basement,Sys Out Temp.
2020-12-14 07:32:22-05:00,house.kitchen,True,20.78889,53.88724,2020-12-14 07:32:22-05:00,35.1,22.0,18.0,22.0,NaN,NaN,NaN,NaN,NaN
2020-12-14 07:32:25-05:00,house.kitchen,False,20.78889,53.89944,2020-12-14 07:32:25-05:00,35.1,22.0,18.0,22.0,NaN,NaN,NaN,NaN,NaN
2020-12-14 07:32:27-05:00,house.kitchen,True,20.80224,53.89944,2020-12-14 07:32:27-05:00,34.9,22.0,18.0,22.0,NaN,NaN,NaN,NaN,NaN
2020-12-14 07:32:30-05:00,house.kitchen,False,20.80224,53.92386,2020-12-14 07:32:30-05:00,34.9,22.0,18.0,22.0,NaN,NaN,NaN,NaN,NaN
2020-12-14 07:32:50-05:00,house.kitchen,True,20.74617,54.16648,2020-12-14 07:32:50-05:00,35.7,22.0,18.0,22.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-08 11:51:24-05:00,house.basement,NaN,NaN,NaN,2021-01-08 11:51:24-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.98,19.98
2021-01-08 11:56:40-05:00,house.basement,NaN,NaN,NaN,2021-01-08 11:56:40-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.91,19.91
2021-01-08 12:01:53-05:00,house.basement,NaN,NaN,NaN,2021-01-08 12:01:53-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.83,19.83
2021-01-08 12:07:04-05:00,house.basement,NaN,NaN,NaN,2021-01-08 12:07:04-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.75,19.75


In [3]:
import pandas as pd
from yadt import scan_and_apply_tz
import json
from datetime import datetime
from thermostat_aggregation_utils import aggregate_to_dataframe, metric_str_to_json

from tqdm import tqdm

from os import listdir
from os.path import isfile, join


#onlyfiles = [f for f in listdir('dump') if isfile(join('dump', f))]
#onlyfiles.reverse()
onlyfiles = [f for f in listdir('dump') if isfile(join('dump', f)) and (f.startswith('thermostat-') or f.startswith('environment-sensor_'))]
#print(f"Files to process : {len(onlyfiles)}")

thermostat_metric_data = pd.DataFrame()
exist = 0
processed = 0
for f in tqdm(onlyfiles):

    with open('dump/'+f, 'r') as file:
        data = file.read().replace('\n', '')
        #print(f)
        try:
            last_json = metric_str_to_json(data)
            
            merge, thermostat_metric_data = aggregate_to_dataframe(f,last_json, thermostat_metric_data)


            if merge:
                #print(f"Import {f}")
                processed = processed + 1
            else:
                #print(f"PASS - {f}")
                exist = exist+1
            
            #if exist > 200:
                #break
            
        except Exception as e:
            raise e
            #else:
            #    pass
            #print(data)

thermostat_metric_data
        

c:\Users\T834432\Documents\personnel\thermostat\thermostat-agent\.venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 116296/116296 [2:48:43<00:00, 11.49it/s]


,location,stove_exhaust_temp,dt,Coil Power,filename,motion,temperature,humidity,PPD,Indoor Temp. Setpoint,MA Temp.,Htg SP
2020-12-09 02:29:06-05:00,house.basement.stove,16.0625,2020-12-09 02:29:06-05:00,0.0,thermostat-20201210-020810,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-04 08:39:50-05:00,house.kitchen,0.0000,2020-12-04 08:39:50-05:00,NaN,thermostat-20201210-020817,True,21.62470,49.51858,24.8,22.0,18.0,22.0
2020-12-04 08:44:17-05:00,house.kitchen,NaN,2020-12-04 08:44:17-05:00,NaN,thermostat-20201210-020819,False,21.80896,49.63913,22.5,22.0,18.0,22.0
2020-12-04 08:49:28-05:00,house.kitchen,NaN,2020-12-04 08:49:28-05:00,NaN,thermostat-20201210-020824,False,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-04 08:49:50-05:00,house.kitchen,NaN,2020-12-04 08:49:50-05:00,NaN,thermostat-20201210-020825,True,22.10803,50.92393,18.8,22.0,18.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-09 07:34:14-05:00,house.kitchen,NaN,2021-01-09 07:34:14-05:00,NaN,thermostat-20210109-123414,False,18.76211,42.97246,72.6,22.0,18.0,22.0
2021-01-09 07:37:56-05:00,house.basement.stove,17.8125,2021-01-09 07:37:56-05:00,0.0,thermostat-20210109-123757,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-09 07:39:14-05:00,house.kitchen,NaN,2021-01-09 07:39:14-05:00,NaN,thermostat-20210109-123914,False,18.73274,42.96941,73.1,22.0,18.0,22.0
2021-01-09 07:42:57-05:00,house.basement.stove,17.8125,2021-01-09 07:42:57-05:00,0.0,thermostat-20210109-124258,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#thermostat_metric_data['location'] = thermostat_metric_data.apply(lambda x : x['site'] if pd.isna(x['location']) else x['location'], axis = 1)
#thermostat_metric_data = thermostat_metric_data[~thermostat_metric_data.duplicated(keep='first')]
thermostat_metric_data.sort_index(inplace=True)
thermostat_metric_data

c:\Users\T834432\Documents\personnel\thermostat\thermostat-agent\.venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location,stove_exhaust_temp,dt,Coil Power,filename,motion,temperature,humidity,PPD,Indoor Temp. Setpoint,MA Temp.,Htg SP
2020-12-04 08:39:19-05:00,house.kitchen,NaN,2020-12-04 08:39:19-05:00,NaN,thermostat-20201210-020913,True,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-04 08:39:50-05:00,house.kitchen,0.0000,2020-12-04 08:39:50-05:00,NaN,thermostat-20201210-020817,True,21.62470,49.51858,24.8,22.0,18.0,22.0
2020-12-04 08:43:36-05:00,house.kitchen,0.0000,2020-12-04 08:43:36-05:00,NaN,thermostat-20201210-021017,False,21.79293,49.42245,22.8,22.0,18.0,22.0
2020-12-04 08:44:17-05:00,house.kitchen,NaN,2020-12-04 08:44:17-05:00,NaN,thermostat-20201210-020819,False,21.80896,49.63913,22.5,22.0,18.0,22.0
2020-12-04 08:44:30-05:00,house.kitchen,NaN,2020-12-04 08:44:30-05:00,NaN,thermostat-20201210-021224,True,21.80896,48.63661,22.8,22.0,18.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-09 07:34:14-05:00,house.kitchen,NaN,2021-01-09 07:34:14-05:00,NaN,thermostat-20210109-123414,False,18.76211,42.97246,72.6,22.0,18.0,22.0
2021-01-09 07:37:56-05:00,house.basement.stove,17.8125,2021-01-09 07:37:56-05:00,0.0,thermostat-20210109-123757,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-09 07:39:14-05:00,house.kitchen,NaN,2021-01-09 07:39:14-05:00,NaN,thermostat-20210109-123914,False,18.73274,42.96941,73.1,22.0,18.0,22.0
2021-01-09 07:42:57-05:00,house.basement.stove,17.8125,2021-01-09 07:42:57-05:00,0.0,thermostat-20210109-124258,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
thermostat_metric_data[thermostat_metric_data['location'] == 'house.basement']


In [ ]:
onlyfiles = [f for f in listdir('dump') if isfile(join('dump', f)) and f.startswith('environment')]
x = onlyfiles.index('environment_sensor_basement-20201220-151228')
del onlyfiles[:x]
len(onlyfiles)

In [3]:
%load_ext autoreload
%autoreload 2

c:\Users\T834432\Documents\personnel\thermostat\thermostat-agent\.venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
pickle_dump = pickle.dumps(thermostat_metric_data)
b = bucket.blob('_thermostat_metric_data.p')
b.temporary_hold = False
b.patch()
b.upload_from_string(data=pickle_dump, content_type='text/plain')


c:\Users\T834432\Documents\personnel\thermostat\thermostat-agent\.venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

pickle_dump = pickle.dumps(onlyfiles)
b = bucket.blob('thermostat_metric_list.p')

b.upload_from_string(data=pickle_dump, content_type='text/plain')

In [ ]:
b = bucket.get_blob('thermostat_metric_list.p')

pickle_load = b.download_as_bytes()
test = pickle.loads(pickle_load)
test

In [ ]:
pd.util.hash_pandas_object(test)

In [ ]:
pd.util.hash_pandas_object(test)
